# Team members
- Bryce Grahn
- Michael Rolle
- Werner de jager
- Abdul Gany Osman
- Lavania Naidoo

# Introduction
The aim of this notebook is undertaking a machine learning investigation on the....

# 1. Imports and installations

In [ ]:
#PIP
!pip install plotly
!pip install sweetviz

In [ ]:

# imports
import os
import math
import calendar
import numpy as np
import scipy as sp
import pandas as pd
import sympy as sym
import seaborn as sns
import sklearn as sk
import tensorflow as tf
import tensorflow_decision_forests as tfdf
import matplotlib.pyplot as plt
import sweetviz as sw

from datetime import datetime

from sklearn.model_selection import train_test_split

# define plt settings 
plt.rcParams["font.size"] = 20 
plt.rcParams["axes.labelsize"] = 20 
plt.rcParams["xtick.labelsize"] = 20 
plt.rcParams["ytick.labelsize"] = 20 
plt.rcParams["legend.fontsize"] = 20 
plt.rcParams["figure.figsize"] = (20,10)

# define seaborn settings seaborn
sns.set(style="ticks", color_codes=True)
sns.set_palette("husl")

## Import Dataset

In [ ]:
url = '/kaggle/input/payments-hackathon-rojones/Payments Fraud DataSet/'

customers_df = pd.read_csv(url + "customers.csv", index_col = 'CUSTOMER_ID')
terminals_df = pd.read_csv(url + "terminals.csv", index_col = 'TERMINAL_ID')
merchants_df = pd.read_csv(url + "merchants.csv", index_col = 'MERCHANT_ID')
transactions_test_df = pd.read_csv(url + "transactions_test.csv", index_col = 'TX_ID')


# Remove last row in the train dataset to resolve multiple type issue in IS_RECURRING_TRANSACTION column
transactions_train_df = pd.read_csv(url + "transactions_train.csv", index_col = 'TX_ID', dtype={'IS_RECURRING_TRANSACTION': str})
transactions_train_df = transactions_train_df[transactions_train_df['IS_RECURRING_TRANSACTION'] != 'Fals']
transactions_train_copy = transactions_train_df
transactions_train_df['IS_RECURRING_TRANSACTION'] = transactions_train_df['IS_RECURRING_TRANSACTION'].astype(bool)

# 2. Understand Data

In [ ]:
print("Customers dataset:")
print("----------------------")
display(customers_df.info())
print("\nterminals dataset:")
print("----------------------")
display(terminals_df.info())
print("\nmerchants dataset:")
print("----------------------")
display(merchants_df.info())
print("\ntransactions_test dataset:")
print("----------------------")
display(transactions_test_df.info())
print("\ntransactions_train dataset:")
print("----------------------")
display(transactions_train_df.info())
print("\ninput dataset:")
print("----------------------")

In [ ]:
print("Customers dataset:")
print("----------------------")
display(customers_df.head())
print("\nterminals dataset:")
print("----------------------")
display(terminals_df.head())
print("\nmerchants dataset:")
print("----------------------")
display(merchants_df.head())
print("\ntransactions_test dataset:")
print("----------------------")
display(transactions_test_df.head())
print("\ntransactions_train dataset:")
print("----------------------")
display(transactions_train_df.head())

# 3. Data Preparation

## Merge Datasets


In [ ]:
merged_train_df = pd.merge(transactions_train_df, merchants_df, on='MERCHANT_ID') 
merged_train_df = pd.merge(merged_train_df, customers_df, on='CUSTOMER_ID')
merged_train_df = pd.merge(merged_train_df, terminals_df, on='TERMINAL_ID')
display(merged_train_df.info())

merged_test_df = pd.merge(transactions_test_df, merchants_df, on='MERCHANT_ID')
merged_test_df = pd.merge(merged_test_df, customers_df, on='CUSTOMER_ID')
merged_test_df = pd.merge(merged_test_df, terminals_df, on='TERMINAL_ID')
display(merged_test_df.info())

# DON'T FORGET TO REMOVE THIS FOR FINAL TRAINING
np.random.seed(42)
merged_train_df = merged_train_df.sample(frac=0.3)
display(merged_train_df.shape)

## Check for Column Differences Between Train and Test Data

In [ ]:
# Get the column names of both DataFrames
train_columns = set(merged_train_df.columns)
test_columns = set(merged_test_df.columns)

# Find the columns that are in the train set but not in the test set, and vice versa
train_only_columns = train_columns - test_columns
test_only_columns = test_columns - train_columns

# Print the columns that are different
print("Columns in train but not in test:", train_only_columns)
print("Columns in test but not in train:", test_only_columns)

## Handle Null Values

In [ ]:
# Count the number of null values per column
null_counts_train = merged_train_df.isnull().sum()
null_counts_test = merged_test_df.isnull().sum()

print(f"Number of columns with null data in training set: {len(null_counts_train[null_counts_train > 0].tolist())}")
print(f"Number of columns with null data in test set: {len(null_counts_test[null_counts_test > 0].tolist())}")

# Plot null counts for training set
plt.figure(figsize=(10, 6))
null_counts_train[null_counts_train > 0].plot(kind='bar')
plt.title('Null Counts per Column (Training)')
plt.xlabel('Columns')
plt.ylabel('Null Counts')
plt.show()

# Plot null counts for test set
plt.figure(figsize=(10, 6))
null_counts_test[null_counts_test > 0].plot(kind='bar')
plt.title('Null Counts per Column (Test)')
plt.xlabel('Columns')
plt.ylabel('Null Counts')
plt.show()

In [ ]:
# Calculate the percentage of null values per column
null_percentage_train = (merged_train_df.isnull().sum() / len(merged_train_df)) * 100
null_percentage_test = (merged_test_df.isnull().sum() / len(merged_test_df)) * 100

# Filter to only show columns with null values
null_percentage_train_filtered = null_percentage_train[null_percentage_train > 0]
null_percentage_test_filtered = null_percentage_test[null_percentage_test > 0]

print(f"Number of columns with null data in training set: {len(null_percentage_train_filtered)}")
print(f"Number of columns with null data in test set: {len(null_percentage_test_filtered)}")

# Plot percentage of null values for training set
plt.figure(figsize=(10, 6))
null_percentage_train_filtered.plot(kind='bar')
plt.title('Percentage of Null Values per Column (Training)')
plt.xlabel('Columns')
plt.ylabel('Percentage of Null Values')
plt.show()

# Plot percentage of null values for test set
plt.figure(figsize=(10, 6))
null_percentage_test_filtered.plot(kind='bar')
plt.title('Percentage of Null Values per Column (Test)')
plt.xlabel('Columns')
plt.ylabel('Percentage of Null Values')
plt.show()


## Add Additional Columns


In [ ]:
merged_train_df['FAILURE'] = np.where(merged_train_df['FAILURE_CODE'].isna(), 0, 1)
merged_test_df['FAILURE'] = np.where(merged_test_df['FAILURE_CODE'].isna(), 0, 1)
merged_test_df['DISTANCE_FROM_TERMINAL'] = np.sqrt((merged_test_df.x_customer_id - merged_test_df.x_terminal_id)**2 + (merged_test_df.y_customer_id - merged_test_df.y_terminal__id)**2)['DISTANCE_FROM_TERMINAL'] = np.sqrt((merged_test_df.x_customer_id - merged_test_df.x_terminal_id)**2 + (merged_test_df.y_customer_id - merged_test_df.y_terminal__id)**2)
merged_train_df['DISTANCE_FROM_TERMINAL'] = np.sqrt((merged_train_df.x_customer_id - merged_train_df.x_terminal_id)**2 + (merged_train_df.y_customer_id - merged_train_df.y_terminal__id)**2)['DISTANCE_FROM_TERMINAL'] = np.sqrt((merged_train_df.x_customer_id - merged_train_df.x_terminal_id)**2 + (merged_train_df.y_customer_id - merged_train_df.y_terminal__id)**2)

## Drop Failure Columns

In [ ]:
merged_test_df = merged_test_df.drop(['FAILURE_CODE', 'FAILURE_REASON'], axis=1)
merged_train_df = merged_train_df.drop(['FAILURE_CODE', 'FAILURE_REASON'], axis=1)

# 4. Visualize Data

In [ ]:
# dataset_report = sw.analyze(merged_train_df, "TX_AMOUNT")
# dataset_report.show_notebook(layout='vertical')

# 5. Manipulate Data
## Combine Data for Simultaneous Preprocessing

In [ ]:
# Array of Ids for submission
testIds = merged_test_df.index

# Combine train and test data for simultaneous data preparation
data = pd.concat([merged_train_df.drop('TX_FRAUD', axis=1), merged_test_df.drop('ID_JOIN', axis=1)], keys=['train', 'test'])

display(data.shape)

## View Stats on the Data

In [ ]:
data.describe()

## Convert Dates to Datetimes

In [ ]:
def card_expiry_to_date(expiry_str):
    parts = expiry_str.split('/')
    month = int(parts[0])
    year = int(parts[1])
    if len(str(year)) == 2:
        year += 2000
    last_day = calendar.monthrange(year, month)[1]
    return datetime(year, month, last_day)

# Apply the conversion to CARD_EXPIRY_DATE
data['CARD_EXPIRY_DATE'] = data['CARD_EXPIRY_DATE'].apply(card_expiry_to_date)

# List of date columns
dates = ['FOUNDATION_DATE', 'ACTIVE_FROM', 'TRADING_FROM', 'CARD_EXPIRY_DATE']

# Ensure all date columns are in datetime format
for col in dates:
    data[col] = pd.to_datetime(data[col])

## Splitting the Dataset into Categorical and Numerical Features

In [ ]:
numerical_cols = data.select_dtypes(include=['int64', 'float64', 'datetime'])
categorical_cols = data.select_dtypes(include=['object', 'bool'])
print('Numerical columns:')
print(numerical_cols.columns)
print('Categorical columns:')
print(categorical_cols.columns)

## Get Unique Values Per Categorical Column

In [ ]:
# Get the number of unique values in each categorical column
unique_values_per_column = categorical_cols.nunique()

# Display the result
print(unique_values_per_column)

## Encode Categorical Data

In [ ]:
# data = pd.get_dummies(data)
# display(data.info())

# 6. Split Data
## Split out Combined Train and Test Data

In [ ]:
x_test_final = data.loc['test']
x_total = data.loc['train']
y_total = merged_train_df[['TX_FRAUD']]
display(x_test_final.shape)
display(x_total.shape)
display(y_total.shape)

## Split Train Data Into Train and Test for Validation

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    x_total, y_total, test_size=0.1, random_state=0)
print("x_train:")
display(x_train.head())
print("x_test:")
display(x_test.head())
print("y_train")
display(y_train.head())
print("y_test")
display(y_test.head())